In [1]:
from tokenization_kobert import KoBertTokenizer
from transformers import BertTokenizer, BertForQuestionAnswering, AdamW, BertModel, get_linear_schedule_with_warmup

tokenizer = KoBertTokenizer.from_pretrained("monologg/kobert")

model = BertForQuestionAnswering.from_pretrained("C:\\Users\\tjdgu\\Desktop\\train#1_KorQuAD")

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertTokenizer'. 
The class this function is called from is 'KoBertTokenizer'.


In [2]:
import torch

In [3]:
print(tokenizer.tokenize("소아에서 가장 흔하게 발생하는 피부 감염증은?"))

['▁소', '아', '에서', '▁가장', '▁', '흔', '하게', '▁발생', '하는', '▁피부', '▁감염', '증', '은', '?']


In [4]:
def answer_question(question, answer_text):
    '''
    Takes a `question` string and an `answer_text` string (which contains the
    answer), and identifies the words within the `answer_text` that are the
    answer. Prints them out.
    '''
    # ======== Tokenize ========
    # Apply the tokenizer to the input text, treating them as a text-pair.
    input_ids = tokenizer.encode(question, answer_text)

    # Report how long the input sequence is.
    print('Query has {:,} tokens.\n'.format(len(input_ids)))

    # ======== Set Segment IDs ========
    # Search the input_ids for the first instance of the `[SEP]` token.
    sep_index = input_ids.index(tokenizer.sep_token_id)

    # The number of segment A tokens includes the [SEP] token istelf.
    num_seg_a = sep_index + 1

    # The remainder are segment B.
    num_seg_b = len(input_ids) - num_seg_a

    # Construct the list of 0s and 1s.
    segment_ids = [0]*num_seg_a + [1]*num_seg_b

    # There should be a segment_id for every input token.
    assert len(segment_ids) == len(input_ids)

    # ======== Evaluate ========
    # Run our example through the model.
    outputs = model(torch.tensor([input_ids]), # The tokens representing our input text.
                    token_type_ids=torch.tensor([segment_ids]), # The segment IDs to differentiate question from answer_text
                    return_dict=True) 

    start_scores = outputs.start_logits
    end_scores = outputs.end_logits

    # ======== Reconstruct Answer ========
    # Find the tokens with the highest `start` and `end` scores.
    answer_start = torch.argmax(start_scores)
    answer_end = torch.argmax(end_scores)

    # Get the string versions of the input tokens.
    tokens = tokenizer.convert_ids_to_tokens(input_ids)

    # Start with the first token.
    answer = tokens[answer_start]

    # Select the remaining answer tokens and join them with whitespace.
    for i in range(answer_start + 1, answer_end + 1):
        
        # If it's a subword token, then recombine it with the previous token.
        if tokens[i][0:1] == '▁':
            answer += tokens[i][1:]
            
        # Otherwise, add a space then the token.
        else:
            answer += ' ' + tokens[i]
            
    if answer[-1] == '은' or answer[-1] == '는' or answer[-1] == '이' or answer[-1] == '가' or answer[-1] == '과' or answer[-1] == '의':
        answer = answer[:-1]

    print('Answer: "' + answer + '"')

In [5]:
import textwrap

# Wrap text to 80 characters.
wrapper = textwrap.TextWrapper(width=80) 

bert_abstract = "농가진 또는 농피증은 소아에서 가장 흔한 피부 감염증입니다. 황색 포도알균 또는 A군 사슬알균 등의 세균이 원인입니다. 주로 피부에 상처가 생기거나 벌레에 물린 후에 발생합니다. 작은 붉은 반점으로 시작해 얇은 수포나 농포가 생긴 후 터집니다. 터진 수포에서 노란 진물이 나오고 딱지가 앉으며, 때로 가려움이 동반되지만 통증이나 전신적인 발열은 거의 없습니다. 손이나 옷을 통해 피부의 다른 부위로 번질 수 있습니다. 피부 병변 부위를 깨끗이 씻어내고 항생제 연고를 국소적으로 발라줍니다."

print(wrapper.fill(bert_abstract))

농가진 또는 농피증은 소아에서 가장 흔한 피부 감염증입니다. 황색 포도알균 또는 A군 사슬알균 등의 세균이 원인입니다. 주로 피부에 상처가
생기거나 벌레에 물린 후에 발생합니다. 작은 붉은 반점으로 시작해 얇은 수포나 농포가 생긴 후 터집니다. 터진 수포에서 노란 진물이 나오고
딱지가 앉으며, 때로 가려움이 동반되지만 통증이나 전신적인 발열은 거의 없습니다. 손이나 옷을 통해 피부의 다른 부위로 번질 수 있습니다. 피부
병변 부위를 깨끗이 씻어내고 항생제 연고를 국소적으로 발라줍니다.


In [6]:
question = "소아에서 가장 흔하게 발생하는 피부 감염증은?"
answer_question(question, bert_abstract)

Query has 176 tokens.

Answer: "▁농 가 진또는농 피 증"


In [7]:
question = "뇌진탕의 휴유증의 지속 기간은?"
bert_abstract = "사전적인 의미로의 뇌진탕은 구조변화 없이 기능적인 소실만이 잠시 일시적으로 나타났다 회복되는 것이므로 외상과 연관된 후유증은 없는 가벼운 상태로 규정되어 왔습니다. 그러나 엄밀한 의미로 구조적인 손상이 전혀 없다고 하기는 어렵다는 것이 최근의 견해입니다. 의식 소실의 시간이나 기억상실의 시간이 길다면 영구적인 기능 감소가 발생할 소지가 있는 것입니다. 가장 흔히 관찰되는 후유증은 뇌진탕 후 증후군(Post-Concussion syndrome)이라고 하며 주 증상으로는 두통 및 뇌신경관련 증상으로 어지럼증, 귀울림(이명), 청력감퇴, 시력장애 등을 호소합니다. 정신과적 증상으로 과민, 불안, 우울, 인격변화, 피로, 수면장애, 인지장애, 기억장애, 집중력 및 주의력 장애 등을 호소합니다. 대부분의 경우 3개월 내에 증상이 없어지지만, 소수의 경우 1년 이상 지속될 수 있습니다."
answer_question(question, bert_abstract)

Query has 266 tokens.

Answer: "▁3 개월"


In [8]:
question = "만성 기침 지속 기간은?"
bert_abstract = "기침은 원인이 다양하기 때문에 원인 진단을 위해서는 기침의 지속기간에 따른 분류가 중요합니다.일반적으로 급성 기침은 지속 기간이 3주 이내이며, 그 원인으로는 감기, 알레르기성 비염, 급성 세균성 기관지염, 만성폐쇄성폐질환의 급성 악화 등이 있습니다. 아급성 기침은 3주에서 8주 정도 지속되는 기침으로, 그 원인으로는 감염 후 기침(post-infectious cough), 세균성 부비동염, 천식 등이 있습니다. 만성 기침은 8주 이상 지속되는 기침으로 상기도기침증후군(upper airway cough syndrome), 기침 이형 천식(cough variant asthma), 위식도역류질환(gastroesophageal reflux disease, GERD)가 흔한 3대 원인이지만 만성 기침의 원인을 찾을 수 없는 경우도 많습니다."
answer_question(question, bert_abstract)

Query has 272 tokens.

Answer: "▁8 주"


In [9]:
question = "환청은 어떤 정신 질환에서 나타나나?"
bert_abstract = "이명이란 실제로 외부에서 나는 소리가 없음에도 머리나 귀에서 소리가 들린다고 느끼는 증상입니다. 환자들은 '윙~', '쐬~'하는 소리, 매미 우는 소리, 바람 소리로 표현하는 경우가 많고, 일부는 이런 소리가 혼합되어 들린다고 호소합니다. 이명은 타각적 이명(objective tinnitus)과 자각적 이명(subjective tinnitus)으로 구분합니다. 타각적 이명은 혈류나 근육의 경련 같은 체내의 소리가 몸을 통해 귀에 전달되어 들리며, 검사자도 그러한 소리를 들을 수 있습니다. 자각적 이명은 검사자는 들을 수 없고 환자 본인만 소리가 들린다고 느끼는 경우를 말합니다. 자각적 이명은 환청과는 다릅니다. 환청은 음악이나 목소리처럼 의미가 있는 소리가 들리는 증상이며, 조현병과 같은 정신질환에서 나타납니다."
answer_question(question, bert_abstract)

Query has 241 tokens.

Answer: "▁조 현 병"


In [10]:
question = "만성적으로 재발하는 가려움증을 동반하는 피부염은?"
bert_abstract = "아토피피부염은 만성적으로 재발하는 가려움증을 동반하는 피부염으로, 영유아기에 흔히 발생하고 환자나 가족 중에 아토피천식, 알레르기 비염 같은 알레르기 질환을 동반하는 경우가 많습니다. 유아기의 아토피피부염은 초기에는 뺨이나 이마, 머리에 잘 생기는데, 몸통이 거칠고 건조하며, 팔다리는 접히는 부위보다는 팔다리의 바깥쪽에 피부염이 있는 경우가 많습니다. 유아기에는 진물이나 가피(딱지)가 흔히 나타나기도 합니다. 소아기의 특징은 피부 병변이 얼굴은 오히려 덜 침범되는데 반하여 팔오금, 목 같이 접히는 부위는 후기로 갈수록 침범이 뚜렷해지며, 건조한 형태로 나타나는 경우가 많습니다. 사춘기와 성인기 때는 오랫동안 긁어 피부가 두껍게 보이는 현상(태선화)이 뚜렷하게 나타나며, 목 같은 접히는 부위 뿐 아니라 얼굴이나 손도 흔히 침범됩니다. 아토피피부염 환자의 50% 이상이 생후 3개월에서 1년 이내에 발병하고, 30%가 1년에서 5년 사이에 발병합니다. 즉 5세 이전에 발병하는 예가 대부분입니다. 환자의 80%는 소년기 중 천식이나 비염이 발생할 수 있고, 호흡기계 알레르기가 발생하면 피부증상은 호전되는 경우가 흔합니다. 아토피피부염은 대체로 유소아에서 증상이 더 심하고 지속적이다가 나이가 들면서 호전되는 경향이 있습니다. 즉, 2-3년 사이에 80% 정도는 증상이 좋아지지만 간혹 성인이 되어서도 지속되는 경우가 있습니다."
answer_question(question, bert_abstract)

Query has 417 tokens.

Answer: "▁아 토 피 피 부 염"


In [11]:
question = "다운 증후근은 21번 염색체가 몇개 있나?"
bert_abstract = "다운 증후군은 21번 염색체가 3개 있는 경우로, 성장 장애와 관련된 가장 흔한 염색체 이상 질환입니다. 출생아 700~800명당 1명의 발생 빈도를 보입니다. 출생 체중은 정상보다 500 g 정도 적고, 출생 신장도 2~3 cm 정도 작습니다. 출생 후에도 성장 장애가 지속됩니다. 사춘기 발달은 평균보다 약간 빠르지만, 사춘기 급성장이 충분하지 않아 최종 성인 키는 작습니다. 터너 증후군은 여성에서 X염색체의 결손에 의해 발생하는 질환입니다. 키가 작으며 생식샘 장애로 사춘기의 발달이 제대로 이루어지지 않습니다. 날개 모양의 목, 방패 모양의 가슴, 밖굽이팔꿈치(바깥쪽으로 구부러진 팔꿈치) 등이 나타날 수 있습니다. X염색체 결손에 의한 성장 관련 유전자의 부족에 의해서 저신장이 생기는 것으로 알려져 있습니다. 우리나라 터너 증후군 환자의 최종 성인 키는 140 cm 정도입니다."
answer_question(question, bert_abstract)

Query has 270 tokens.

Answer: "▁3 개"


In [12]:
question = "성장의 환경적인 요인은?"
bert_abstract = "출생 전 태아기에는 세포의 수가 급격히 증가하여 가장 빠른 성장 속도를 보입니다. 출생 후 성장은 세포의 수와 함께 크기가 증가하는 시기입니다. 태어나서 첫 2년은 출생 전에 비해 성장 속도가 감소하지만 비교적 빠르게 자랍니다. 2세 이후부터 사춘기 전까지는 완만한 성장 속도를 보이다가 사춘기가 시작되면 다시 성장 속도가 빨라집니다. 사춘기 후반 무렵에 골성숙이 진행되어 긴 뼈의 성장판이 닫히게 되면 성장이 완료됩니다. 성장은 유전과 환경의 복합적인 요인에 의해 결정됩니다. 대개 70-80%는 유전적인 요인에 의해 결정되고, 20-30%가 환경적인 요인에 의해 영향을 받습니다. 환경적인 요인으로는 출생 후 영양 상태, 질병 유무, 사회 경제적 여건, 정신 사회적 스트레스, 호르몬 노출, 환경 교란 물질 노출 등 여러 가지가 있습니다."
answer_question(question, bert_abstract)

Query has 223 tokens.

Answer: "▁유 전 적인요인"


In [13]:
question = "오래된 적혈구의 파괴로 인해 생성되는 것은?"
bert_abstract = "정상적으로 대부분의 빌리루빈은 오래된 적혈구의 파괴로 인해 생성됩니다. 적혈구가 분해되어 생긴 빌리루빈이 간세포에서 물에 녹을 수 있는 수용성의 결합형 빌리루빈으로 변합니다. 이후 담즙에 녹아 소화 기관인 장으로 배설되어 대부분은 대변으로 배출됩니다. 결합형 빌리루빈의 일부는 장내 세균에 의해 유로빌리노젠으로 변하여 장에서 재흡수되어 담즙과 소변으로 배설됩니다. 소변에서 빌리루빈이 많이 검출된다면 혈액 속에 결합형 빌리루빈이 많이 증가되어 있음을 뜻하며 간이나 담도에 질환이 있을 가능성이 매우 높습니다. "
answer_question(question, bert_abstract)

Query has 179 tokens.

Answer: "▁빌 리 루 빈"


In [14]:
question = "두경부에서 후두암보다 발생 빈도가 높은 것은?"
bert_abstract = "후두암은 두경부에서 갑상선암 다음으로 발생 빈도가 높습니다. 2020년에 발표된 국가암 등록통계에 따르면 2018년 우리나라 암 발생 총 243,837건 중 후두암은 1,143건으로 전체 암의 0.5%를 차지했습니다. 인구 10만명당 조발생률은 2.2명으로 보고되었습니다(보건복지부 중앙암 등록본부 2020년 12월 발표 자료). 흡연과 음주가 주요 위험인자입니다. 흡연량에 따라 후두암 발생 위험이 3~44배 증가하며, 금연하면 위험이 1/3 정도로 줄어듭니다. 음주는 음주량에 따라 후두암 발생 위험이 1.4~5.9배 증가하며, 흡연과 동시에 음주하는 경우 위험이 더 커집니다. 또 다른 위험 인자로 사람유두종바이러스(human papilloma virus, HPV)가 거론되고 있으나 아직 논란이 많습니다. 쉰소리는 후두암 진단에 가장 중요한 증상입니다. 발성 기관인 성대에 암이 발생하면 음성이 변하므로, 이를 통해 조기 진단할 수 있습니다. 그러나 성대의 상부나 하부에 암이 생기면 초기에 음성 변화가 없을 수 있습니다. 이 부위에 생긴 암의 초기 증상은 인후부의 막연한 불편감 또는 연하 시 이물감 등입니다. 40세 이상의 남자에서 특별한 원인 없이 쉰소리가 2주 이상 지속된다면 후두 진찰이 필수적이며, 그 외의 증상으로 호흡곤란과 천명(음)(천식 등의 질환으로 숨을 내쉴 때 쌕쌕 소리가 나는 증상), 통증, 연하곤란, 기침, 각혈, 체중감소, 구취, 경부종물, 압통 등이 나타날 수 있습니다."
answer_question(question, bert_abstract)

Query has 425 tokens.

Answer: "▁ 갑 상 선 암"


In [15]:
question = "폐경이 발생하면 어떤 (호르몬) 분비가 감소하는가?"
bert_abstract = "폐경의 원인은 다음과 같습니다. 1. 정상적인 난소의 퇴화.나이가 들면서 정상적인 노화에 의해 난소가 퇴화하면 여성호르몬의 분비가 현저하게 감소해 폐경이 발생합니다. 2. 외과적인 손상.많은 여성이 암이나 양성 질환 등 다양한 원인으로 자궁이나 난소를 절제합니다. 폐경 전에 양쪽 난소를 모두 제거하면 몸속에 여성호르몬을 생성하는 생식샘이 없어지므로 폐경을 맞게 됩니다. 한편, 한쪽 또는 양쪽 난소를 남겨둔 상태로 자궁만 절제할 경우에는 월경혈의 생산 장소인 자궁이 없어져 월경이 멈추지만 여성호르몬을 생성하는 난소는 남아 있기 때문에 몸의 호르몬 환경은 폐경 전과 동일합니다. 3. 항암치료. 수술로 난소를 제거하지 않아도 암으로 인해 항암제를 투여받거나 골반 부위에 방사선치료를 받을 경우 난소가 손상을 입어 폐경이 발생할 수 있습니다. 자가면역질환, 볼거리 같은 난소의 염증, 지나친 다이어트 등 영양부족이 심한 경우, 흡연, 유전적 소인이 있는 여성에서 조기폐경이 발생할 위험이 높습니다."
answer_question(question, bert_abstract)

Query has 315 tokens.

Answer: "▁여성 호 르 몬"


In [16]:
question = "약국에서 처방전 없이 살 수 있는 수면제를 다량 복용했을 시 발생하는 합병증은? "
bert_abstract = "약국에서 처방전 없이 쉽게 살 수 있는 수면제는 자살 목적으로 다량 섭취하는 경우가 있습니다. 이러한 수면제에 많이 포함된 주요성분으로는 독실아민(doxylamine)이란 약물이 있습니다. 독실아민은 항히스타민 계통의 수면유도제로 일반적으로는 안전하다고 알려져 있으나 과량 복용하였을 때 사망한 경우도 있고, 횡문근융해증, 경련, 흡인성 폐렴 등의 합병증을 유발할 수 있습니다. "
answer_question(question, bert_abstract)

Query has 153 tokens.

Answer: "▁ 흡 인 성폐 렴등의합병 증"


In [17]:
question = "대향 객혈을 유발하는 질환에 관여하는 것은?"
bert_abstract = "객혈을 초래하는 질환은 40여 종 이상이지만 그 중에서 대량 객혈을 유발하는 질환의 대부분은 폐결핵, 기관지확장증, 폐농양, 만성 기관지염, 폐암, 폐아스페르길루스종 등이 관여하는 것으로 알려져 있습니다. 최근 결핵과 기관지 확장증의 유병률이 감소하는 경향을 보이고 있지만, 아직도 우리나라에서는 이들 질환이 주요한 원인을 차지하고 있습니다. 특히 50대 이상에서는 폐암, 폐결핵, 기관지확장증의 순서로 관찰된다고 알려져 있습니다."
answer_question(question, bert_abstract)

Query has 157 tokens.

Answer: "▁폐 결 핵 ,기관 지 확 장 증 ,폐 농 양 ,만 성기관 지 염 ,폐 암 ,폐 아 스 페 르 길 루 스 종"


In [18]:
question = "설사를 유발하는 기생충들은?"
bert_abstract = "장에 기생하는 기생충 중에 지아르디아, 작은와포자충, 회충, 촌충은 설사를 유발시킬 수 있습니다. 감염된 음식물을 다루는 사람들이 기생충 감염의 위험에 노출되긴 하지만, 기생충은 주로, 오염된 물을 통해 체내에 들어오게 되므로 안전한 물을 마시는 것이 감염을 예방하는 가장 좋은 방법입니다. 크론병과 궤양성 대장염 및 베체트장염은 염증성 장질환에 속하며, 설사가 첫 증상으로 나타날 수 있습니다. 궤양성 대장염은 설사와 함께 혈변이 동반되고 크론병과 베체트장염은 복통이 함께 나타나는 경우가 많습니다. 후천성 면역결핍증이나 암 환자들은 면역계가 약화되어 있어 심한 설사를 보일 수 있습니다. 영양 부족으로 점막의 부종이 있어 흡수장애가 생기면 설사를 할 수 있고 또한 쉽게 감염되어 설사가 나타날 수 있습니다. "
answer_question(question, bert_abstract)

Query has 245 tokens.

Answer: "▁지 아 르 디 아 ,작은 와 포 자 충 ,회 충 , 촌 충"


In [19]:
question = "앞이마 근육이나 눈가 아래 얼굴의 근육에 수축 현상이 동반되는 얼굴떨림은?"
bert_abstract = "눈꺼풀 연축은 눈가 주변 근육이 저절로 수축이 일어나서 눈이 감기는 현상이 일어나는데 반복적으로 짧은 시간 동안 나타나기도 하며, 때로는 지속적으로 나타나기도 합니다. 눈꺼풀에 수축 현상이 일어날 때 앞이마 근육이나 눈가 아래 얼굴의 근육에 수축 현상이 동반되기도 합니다. 반얼굴 연축은 대부분의 경우 한쪽 얼굴에만 나타나기 때문에 반얼굴떨림증이라고 불립니다. 발병 초기에는 눈꺼풀의 경미한 단일수축이 나타나다가 점차 진행하면서 얼굴의 아래편에도 근육의 수축 현상이 나타납니다. "
answer_question(question, bert_abstract)

Query has 168 tokens.

Answer: "▁반 얼 굴 떨 림 증"
